# data preprocessing

In [ ]:
'''
[kx, ky, kz, coil, phase, set, slice, rep, avg], 
frequency encoding, first phase encoding, second phase encoding, 
coil, phase (time), set (velocity encoding), slice, repetition, and number of averages

better not to take average over the repetition dimension
'''

In [ ]:
import os
import read_ocmr as read
import matplotlib.pyplot as plt
import math
## need to install ismrmrd and ismrmrdtools
from ismrmrdtools import show, transform

import numpy as np
import torch

In [ ]:
datapath = '/mnt/shared_a/OCMR/OCMR_data_fully_sampled/' # change this path to your directory
h5files = list([])
for file in os.listdir(datapath):
    if file.endswith(".h5"):
        h5files.append(file)
print('total amount of files:', len(h5files))

In [ ]:
regular_files = []
for file in h5files:
    kData,param = read.read_ocmr(datapath+file)
    print('filename', file, 'Dimension of kData: ', kData.shape)
    curr_shape = kData.shape
    if curr_shape[0]==384 and curr_shape[1]==144:
        regular_files.append([file, curr_shape])

### save these useful files
path = '/home/huangz78/rl_samp/useful_files.npz'  # change this path to your directory
np.savez(path,files=regular_files)

In [ ]:
savepath = '/mnt/shared_a/OCMR/OCMR_fully_sampled_images/' # change this path where you save processed data
for fileind in range(len(regular_files)):
    filename    = regular_files[fileind][0]
    kData,param = read.read_ocmr(datapath+file)
    kData_tmp   = np.mean(kData, axis = 8); # average the k-space if average > 1
    # Image reconstruction (SoS)
    im_coil = transform.transform_kspace_to_image(kData_tmp, [0,1]); # IFFT (2D image)
    im_sos  = np.sqrt(np.sum(np.abs(im_coil) ** 2, 3)) # Sum of Square
    print('Dimension of Image (with ReadOut ovesampling): ', im_sos.shape)
    RO    = im_sos.shape[0]
    image = im_sos[math.floor(RO/4):math.floor(RO/4*3),:,:] # Remove RO oversampling
    print('Dimension of Image (without ReadOout ovesampling): ', image.shape)
    
    for sliceind in range(image.shape[5]):
        image[:,:,:,:,:,sliceind,:] /= image[:,:,:,:,:,sliceind,:].max() # normalization of the image
    
    torch.save(torch.tensor(image) , savepath + filename[:-3] + '.pt')
    print(f'file {savepath + filename} is successfully saved')

# view an example file

In [ ]:
datapath = '/mnt/shared_a/OCMR/OCMR_fully_sampled_images/' # change as needed
file = 'fs_0074_1_5T.pt'  # pick an example file
data = torch.load(datapath+file)

In [ ]:
# Show the reconstructed cine image
from IPython.display import clear_output
import time

slc_idx = 6 # math.floor(SLC/2)
print(slc_idx)
for rep in range(5): # repeat the movie for 5 times
    for frame in range(data.shape[3]):
        clear_output(wait=True)
        plt.imshow(data[:,:,0,frame,0,slc_idx,:], vmin=0, vmax=data.max(), cmap = 'gray'); plt.axis('off');
        plt.colorbar()
        plt.show() 
        time.sleep(0.03)